# Automated trading using Python and OANDA
The purpose of this project is to attempt to create a function that has a basic strategy to buy and sell currency. 


# 1 - Getting historical data.
Using yfinance and pandas I am downloading historical data over the last ~60days in 15 minute intervals for Pound against Dollar.
This will be used to test if the signal function in section 2 is working as intended.

In [3]:
import yfinance as yf
import pandas as pd

dataF = yf.download("GBPUSD=X", start="2023-06-28", end="2023-08-24", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-06-28 00:00:00,1.274843,1.274876,1.274551,1.274843,1.274843,0
2023-06-28 00:15:00,1.274827,1.274892,1.274324,1.274324,1.274324,0
2023-06-28 00:30:00,1.274616,1.274730,1.274616,1.274714,1.274714,0
2023-06-28 00:45:00,1.274681,1.274762,1.274486,1.274584,1.274584,0
2023-06-28 01:00:00,1.274486,1.275120,1.274421,1.275120,1.275120,0
...,...,...,...,...,...,...
2023-08-23 22:45:00,1.272394,1.272394,1.271795,1.271795,1.271795,0
2023-08-23 23:00:00,1.271989,1.272540,1.271989,1.272491,1.272491,0
2023-08-23 23:15:00,1.272669,1.272669,1.272329,1.272346,1.272346,0


# 2 - Defining the signal function
In this function we must identify what the market pattern is in order to allow the bot to decide what to currently do.

In my strategy I will focus only on candle engulfing patterns.

In [4]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [5]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    3508
2     199
1     181
Name: count, dtype: int64

As we can see the function works well as we have 82 sells and 71 buys with the historical data, so we can safely move onto the live API calls.
# 3 - Connecting to API and executing trades
In this section I will be accessing my OANDA account with my access token and attempting execute a schedule for my trading_job function.

In [6]:
%pip install apscheduler
%pip install oandapyV20
%pip install oanda_candles


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [8]:
access_token = '<YOUR-API-ACCESS-TOKEN>'
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.GBP_USD, Gran.M15)
    candles = collector.grab(n)
    return candles
#last 3 candles is the most recent
candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)

True
True
True


In [9]:
def trading_job():
    #last candle open, price is fluctuating for close
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    i=0

    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i += 1
    
    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    #providing the live dataframe into the signal gen function.
    signal = signal_generator(dfstream.iloc[:-1,:])
    
    accountID = "<YOUR-ACCCOUNT-ID>"
    client = API(access_token)

    #stop-loss-take-profit ratio allows us to know when to buy/sell
    SLTPRatio = 2
    #previous candle range open - close
    previous_candleR = abs(dfstream['Open'].iloc[-2]-dfstream['Close'].iloc[-2])

    SLBuy = float(str(candle.bid.o)) -previous_candleR
    SLSell = float(str(candle.bid.o)) +previous_candleR

    #multiplied by SLTP to aim for 2x profit
    TPBuy = float(str(candle.bid.o)) + previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o)) - previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1:])
    print(TPBuy, " ", SLBuy, " ", TPSell, " ", SLSell)
    #signal = 1
    
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument = "GBP_USD", units = -1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data = mo.data)
        rv = client.request(r)
        print(rv)
    
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument = "GBP_USD", units = 1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data = mo.data)
        rv = client.request(r)
        print(rv)

# 4 - Testing the trading function
To test this without having to wait for the scheduler, I will simply set the signal = 1 (sell) and run the trading job function.

In [10]:
trading_job()

      Open    Close     High      Low
0  1.26309  1.26269  1.26339  1.26267
1  1.26269  1.26317  1.26333  1.26262
1.2641099999999996   1.2626700000000002   1.2621900000000004   1.2636299999999998
{'orderCreateTransaction': {'id': '4', 'accountID': '101-004-26679487-001', 'userID': 26679487, 'batchID': '4', 'requestID': '79158111713791040', 'time': '2023-08-24T16:42:15.464984654Z', 'type': 'MARKET_ORDER', 'instrument': 'GBP_USD', 'units': '-1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.26219', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.26363', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '5', 'accountID': '101-004-26679487-001', 'userID': 26679487, 'batchID': '4', 'requestID': '79158111713791040', 'time': '2023-08-24T16:42:15.464984654Z', 'type': 'ORDER_FILL', 'orderID': '4', 'instrument': 'GBP_USD', 'units': '-1000', 'requestedUnits': '-1000', 'price': '1.26274', 'pl'

![Sucessful-Sale-OANDA](https://i.imgur.com/LEKbgpQ.png)

# 5 - Call the trading job function
Using the Blocking Scheduler, I can add the trading function to be called 4 times every hour, mon-fri continuously.
It will take the last 3 candles at each of the 15 minute intervals and determine whether to buy, sell or do nothing depending on what the function returns.

In [11]:
scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week = 'mon-fri', hour='00-23',minute='1,16,31,46',start_date='2023-08-25 12:00:00')
scheduler.start()